# Evaluation run wflow_sbm calibrated

In [1]:
import os
import subprocess
import pandas as pd
import xarray as xr

from glob import glob
from pathlib import Path
from pathos.threading import ThreadPool as Pool

# Set Paths

In [2]:
# Set Paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/')
MODELS = Path(f'{ROOT}/model_parameters/wflow_sbm/')
RESULTS = Path(f"{ROOT}/results/streamflow_evaluation/wflow_sbm_calibration/")

julia_path = '/gpfs/home6/jaerts/julia-1.7.3/bin/julia --threads 4'

# Config

In [3]:
# Get available basin IDs wflow_sbm
calibration_file = f"{RESULTS}/wflow_calibration_objective_function_overview.csv"
df = pd.read_csv(calibration_file, index_col='basin_id')
basin_ids = df.index.to_list()

# Set available cores
cores_available = 20

# Sort basins by size

In [4]:
# Sort by basin size
def sort_basin_ids_by_size(basin_ids):
    sizes = []
    for basin_id in basin_ids:
        size = os.path.getsize(f'{MODELS}/{basin_id}/staticmaps.nc')
        sizes.append(size)

    df = pd.DataFrame()
    df['basin_id'] = basin_ids
    df['size'] = sizes
    df = df.sort_values('size')

    basin_ids = df.basin_id.to_list()
    
    return basin_ids

basin_ids_sorted = sort_basin_ids_by_size(basin_ids)

In [5]:
len(basin_ids_sorted)

646

In [6]:
basin_ids_sorted

[80005,
 31023,
 80004,
 80003,
 39061,
 37033,
 28033,
 27038,
 44006,
 26006,
 27051,
 55008,
 54022,
 39036,
 25003,
 65008,
 27047,
 52016,
 58010,
 41013,
 42017,
 80006,
 39065,
 41001,
 65005,
 48005,
 27073,
 30014,
 41016,
 56007,
 102001,
 36010,
 36009,
 101005,
 27032,
 54060,
 38007,
 107001,
 39017,
 7006,
 48004,
 17015,
 52015,
 39054,
 18014,
 31025,
 44009,
 41028,
 41020,
 36011,
 29009,
 39099,
 60012,
 101002,
 40017,
 20002,
 40021,
 28044,
 25012,
 38012,
 21026,
 41029,
 41027,
 39049,
 19010,
 38014,
 72014,
 40006,
 72007,
 28115,
 49004,
 39015,
 12009,
 44011,
 46005,
 84016,
 40020,
 30015,
 85004,
 54025,
 19020,
 42011,
 63004,
 47009,
 73015,
 39005,
 64006,
 37019,
 23011,
 38017,
 65007,
 27077,
 39095,
 49002,
 29003,
 40033,
 42026,
 54034,
 84020,
 39052,
 21017,
 23016,
 28055,
 38021,
 37031,
 89002,
 47014,
 58008,
 37011,
 88001,
 41022,
 47018,
 74006,
 106001,
 33054,
 58009,
 73009,
 39127,
 28060,
 28039,
 17003,
 73014,
 38026,
 84026,
 520

# Model Run Functions

In [6]:
def wflow_runner_evaluation(julia_path, basin_id):
    print(f'Starting: {basin_id}')
    
    # Set config_file
    config_file = f'{MODELS}/{basin_id}/wflow_sbm_evaluation.toml'
    
    # Call wflow julia command line
    subprocess.call(
                    f'{julia_path} -e "using Wflow; Wflow.run()" {config_file}',
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT,
                    shell=True
                   )
    
    return print(f'Finished: {basin_id}')
    
def parallel_run(julia_path, basin_ids, threads=cores_available):
    
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
        
    # Run parallel models
    pool.map(wflow_runner_evaluation, julia_paths, basin_ids)
    return

# Check if output exists

In [21]:
df = pd.DataFrame()
basins = []
exists = []

for basin_id in basin_ids_sorted:
    basins.append(basin_id)

    # check if file exists
    sim_file = Path(f'{MODELS}/{basin_id}/evaluation/output.csv')
    if sim_file.is_file() is False:
        exists.append(False)
    else:
        df_sim = pd.read_csv(sim_file)
    
        # Check if csv containes output
        if len(df_sim) < 2000:
            exists.append(False)
        else:
            exists.append(True)
        
df['basin_id'] = basins
df['completed'] = exists
df = df.reset_index()
# df = df[df['completed'] == True]

basin_ids_sorted = df.basin_id.to_list()

# Create lists and run function

In [7]:
julia_paths = [julia_path] * len(basin_ids_sorted)
parallel_run(julia_paths, basins_redo, threads=cores_available)

Starting: 25012
Starting: 21026
Starting: 38012
Starting: 41029
Starting: 41027
Starting: 39049
Starting: 19010
Starting: 38014
Starting: 72014
Starting: 40006
Starting: 72007
Starting: 49004
Starting: 28115
Starting: 39015
Starting: 44011
Starting: 12009
Starting: 46005
Starting: 84016
Starting: 40020
Starting: 30015
Starting: 85004
Starting: 19020
Starting: 54025
Starting: 63004
Starting: 42011
Starting: 47009
Starting: 73015
Starting: 39005
Starting: 64006
Finished: 85004
Finished: 19020
Finished: 42011
Finished: 21026
Finished: 46005
Finished: 19010
Finished: 38012
Finished: 28115
Finished: 39015
Finished: 72007
Finished: 47009
Finished: 49004
Finished: 54025
Finished: 64006
Finished: 39005
Finished: 84016
Finished: 63004
Finished: 40006
Finished: 30015
Finished: 25012
Finished: 39049
Finished: 41027
Finished: 72014
Finished: 73015
Finished: 12009
Finished: 41029
Finished: 40020
Finished: 44011
Finished: 38014
